# Parallel Processing in Python
In this tutorial, you will learn how to use the Python builtin Multiprocessing library to parallelize computations. This is often one of the first  step when dealing large computations. We will cover the following:
- Using Multiprocessing with a single parameter function
- Using Multiprocessing with multiple parameters function
- Use joblib library for paralellizing functions
- Other ways to parallelize functions in Python

For more technical details about the multiprocessing library, please see the documentation [here](https://docs.python.org/3/library/multiprocessing.html)

# Python setup

In [ ]:
from multiprocessing import Pool
import asyncio
import time
import aiohttp
import pandas as pd

# Optimizing Pandas

## Read only a subset of the rows

In [ ]:
file_40gb = "/Volumes/GoogleDrive/My Drive/WBG/Mozambique/data-cdrs/mCellCDRs.csv"
# df_first_5mill = pd.read_csv(file_40gb, nrows=5000000)

## Read data in chunks

In [ ]:
# set chunck size to 5 million rows
chunksize = 2*1e6
cnt = 1
tot_rows = 0
with pd.read_csv(file_40gb, chunksize=chunksize) as reader:
    for chunk in reader:
        print("Working on chunk: {}".format(cnt))
        print(chunk.shape)
        tot_rows += chunk.shape[0]
        cnt += 1

## Read only a subset of columns
With pandas, you can also read a subset of the columns instead reading all columns.

### EXERCISE-1: Which arguement do you use to read only a subset of columns in pandas?

# Speeding up IO Bound Programs
Refer to the Python script for this.

## Using multiprocessing with a single parameter function

## Define function we would like to run
For the sake demonstration here, we will define a very simple function.

In [ ]:
def square(num):
    return num**2

## Run the function sequentially on a big list

In [ ]:
result = [square(x) for x in list(range(100000))]

## Run with multiprocessing
These lines create a multiprocessing pool of eight workers, and we can use this pool to map our required function to this list. The Pool class represents a pool of worker processes. It has methods which allows tasks to be offloaded to the worker processes in a few different ways.

In [ ]:
pool = Pool(8)
result = pool.map(square,list(range(100000)))
pool.close()

## Compare the linear/sequential processing with the parallel processing

In [ ]:
def run_func(list_length):
    print("Size of List:{}".format(list_length))
    t0 = time.time()
    result1 = [square(x) for x in list(range(list_length))]
    t1 = time.time()
    diff = round(t1-t0, 4)
    print("Running time-Sequential Processing: {} seconds".format(diff))
    time_without_multiprocessing = diff
    # Run with multiprocessing
    t0 = time.time()
    pool = Pool(8)
    result2 = pool.map(square,list(range(list_length)))
    pool.close()
    t1 = time.time()
    diff = round(t1-t0, 4)
    print("Running time-Multiprocessing: {} seconds".format(diff))
    time_with_multiprocessing = diff
    return time_without_multiprocessing, time_with_multiprocessing

In [ ]:
def main():
    times_taken = []
    for i in range(1, 9):
        list_length = 10**i
        time_seq, time_parallel = run_func(list_length)
        times_taken.append([list_length, 'No Multiproc', time_seq])
        times_taken.append([list_length, 'Multiproc', time_parallel])

    timedf = pd.DataFrame(times_taken,columns = ['list_length', 'type','time_taken'])
    fig =  px.line(timedf,x = 'List-Length',y='Timetaken',color='type',log_x=True)
    plotly.offline.plot(fig, filename='comparison_bw_multiproc.html')

### EXERCISE-2: 
1. Make the function 3 seconds slower and report what happens. **Hint.** You can use ```time.sleep(seconds)``` to simulate a more tie consumning function.
2. Find out the number of logical processors on your machine
3. Reduce the number of processors available to Pool and see if it has effect on the running time.

## Using multiprocessing with multiple parameter function
We can extend the code above to run a function with multiple parameters. We can simply use the ```starmap``` method for ```Pool``` to achieve this as below.

In [ ]:
def square_add(num1, num2):
    return num1**2 + num2**2

In [ ]:
processors = Pool(4)
params = [[100,4],[150,5],[200,6],[300,4]]
results = processors.starmap(square_add, params)
processors.close()
print(results[2])

# Other ways to utilize multiprocessing in Python
There are other libraries which can be used to achieve parallization in Python. FOr example:
1. [concurrent.futures](https://docs.python.org/3/library/concurrent.futures.html). Although its also a builtin library, it is considered more high level and relatively easier to use than using multiprocessing directly.
2. [Joblib](https://joblib.readthedocs.io/en/latest/)